## Pydantic
### 🔍 What is Pydantic?
Pydantic lets you define data structures using Python classes, and it automatically validates and parses data into those structures.

It uses Python type hints to:
- Validate input data (like JSON or dicts)
- Convert types (e.g., string to int, if possible)
- Raise clear, useful errors when data is missing or invalid

### What is it used for?
- **FastAPI:** Pydantic is the backbone for data models (requests, responses).
- **Data Validation:** Ensures incoming data (e.g., from APIs, configs, forms) meets expectations.
- **Parsing JSON or external input:** Converts raw data into well-typed Python objects.
- **Environment/config management:** BaseSettings class lets you manage app settings from env vars, .env files, etc.

In [1]:
!pip install pydantic

In [2]:
from pydantic import BaseModel

Take a look at the example class created below employing two different approaches:

In [3]:
from dataclasses import dataclass

@dataclass
class Person1():
  name:str
  age:int
  city:str

print(Person1(name="Milad", age=35, city="Turin"))

Person1(name='Milad', age=35, city='Turin')


In [4]:
class Person2(BaseModel):
  name:str
  age:int
  city:str

person2 = Person2(name="Milad", age=35, city="Turin")

print(person2)

name='Milad' age=35 city='Turin'


As you can see, there no particular difference in between when printed. So why do you think one would want their class to inherit `BaseModel`?

#### 🚀 Bonus: Pythonic Shortcut — `@dataclass`
Combines both approaches (type annotation only AND `__init__` method) neatly.

Now you can do:

✔️ Auto-generated __init__() \\
✔️ Cleaner syntax \\
✔️ Optional __repr__, __eq__, etc.

**NOTE!** In case there was no `@dataclass` decorator, you wouldn't be able to instantiate objects with values directly similar to when `__init__` method is implemented.

What if we create another onstance of the object `Person2` as you can see below?

In [5]:
person3 = Person2(name="Milad", age=35, city=12)

print(person3)

ValidationError: 1 validation error for Person2
city
  Input should be a valid string [type=string_type, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

**BINGO!**

Yes, you're right! It raises the validation error below:

```
ValidationError: 1 validation error for Person2
```

When you create a class that inherits from `pydantic.BaseModel`, Pydantic automatically validates any data passed to it against the type annotations you’ve defined.

Validation includes:

- Type coercion (e.g., converts "123" to 123 if int is expected)
- Type checking (e.g., raises error if a string is passed where a list is expected)
- Required/optional field checking
- Custom validation logic

#### Optional Fields
In Python, optional fields are attributes that may or may not be provided when creating an object — they’re not required.

There are a few ways to define optional fields depending on whether you're using regular classes, dataclasses, or something like Pydantic. Below is demonstrated an example with Pydantic:

In [6]:
from typing import Optional

class Staff(BaseModel):
    id: int
    name: str
    age: int
    salary: Optional[float] = None    # optional with default value None
    city: Optional[str] = None    # optional with default value None
    is_onboard: Optional[bool] = False    # optional with default value True

In [7]:
staff1 = Staff(id=1, name="Gianluca", city="Rome", age=28)
print(staff1)

id=1 name='Gianluca' age=28 salary=None city='Rome' is_onboard=False


In [8]:
staff2 = Staff(id=1, name="Gianluca", city="Rome", age=28, salary=2500)
print(staff2)

id=1 name='Gianluca' age=28 salary=2500.0 city='Rome' is_onboard=False


What about lists? Do we have to manipulate them differently?

In [9]:
from typing import List

class Classroom(BaseModel):
  room_number: str
  students: List[str]   # list of strings
  teachers: List[str]   # list of strings
  capacity: int

In [11]:
# Now let's create a classroom...
classroom = Classroom(
    room_number="M02",
    students=["Ali", "Samad", "Bruno"],
    teachers=["Gianluca", "Shahdad"],
    capacity=30
)

It seems to be working fine, right?
OK! Now let's try the one below and see if we get an error or not...

In [12]:
# create a different class
classroom2 = Classroom(
    room_number="T05",
    students=["Elena", "John", "Bruno"],
    teachers=("Luca", "Aram"),
    capacity="20"
)

In [14]:
print(classroom2)

room_number='T05' students=['Elena', 'John', 'Bruno'] teachers=['Luca', 'Aram'] capacity=20


**What...!?** \\
As you may have already noticed, the teachers are wrapped in a tuple and yet no validation error occurs. \\
What if we wrap information in a dictionary?

In [16]:
# invalid classroom
classroom3 = Classroom(
    room_number="T05",
    students=["Elena", "John", "Bruno"],
    teachers={"Luca": "Male", "Aram": "Female"},
    capacity="20"
)

ValidationError: 1 validation error for Classroom
teachers
  Input should be a valid list [type=list_type, input_value={'Luca': 'Male', 'Aram': 'Female'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type

In [21]:
try:
  invalid_value = Classroom(room_number="T05",
                            students=["Elena", "John", "Bruno"],
                            teachers=("Luca", "Aram"),
                            capacity="twenty")  # won't raise error if capacity="20"
except ValueError as e:
  print(e)

1 validation error for Classroom
capacity
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='twenty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


### BaseModel (`pydantic.BaseModel`) with Nested Models

Complex structures can be created with nested models.

In [24]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str

class Client(BaseModel):
    name: str
    age: int
    address: Address    # Address class nested

In [26]:
client = Client(
    name="John Doe",
    age=32,
    address=Address(
        street="123 Main St",
        city="Anytown",
        state="CA",
        zip_code="12345"
    )
)

print(client)

name='John Doe' age=32 address=Address(street='123 Main St', city='Anytown', state='CA', zip_code='12345')


### Pydantic Fields: Customization and Constraints

The `Field()` function in Pydantic is used to provide metadata, validation, and configuration for model fields in a Pydantic BaseModel.

In [30]:
from pydantic import BaseModel, Field

class Object(BaseModel):
  name: str=Field(min_length=5, max_length=10)
  price: float=Field(gt=0, lt=1000)
  quantity: int=Field(gt=0, lt=100)
  description: str=Field(None, max_length=300)
  active: bool=True

In [31]:
Object1 = Object(name="Pencil", price=5, quantity=50, description="Pen description")
print(Object1)

name='Pencil' price=5.0 quantity=50 description='Pen description' active=True


In [29]:
Object2 = Object(name="Pen", price=5, quantity=50, description="Pen description")
print(Object2)

ValidationError: 1 validation error for Object
name
  String should have at least 5 characters [type=string_too_short, input_value='Pen', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short

Apparently, according to the error raised, there is something wrong with the object name.



ValidationError:
```
ValidationError: 1 validation error for Object
name
  String should have at least 5 characters [type=string_too_short, input_value='Pen', input_type=str]
```

In [34]:
from pydantic import BaseModel, Field, EmailStr

class User(BaseModel):
  username: str=Field(..., description="Unique username for the user")
  age: int=Field(..., gt=18, description="Age should be greater than 18")
  email: str=Field(..., pattern=r'^[\w\.-]+@[\w\.-]+\.\w+$', description="Email should be a valid email address")  # basic approach
  # email: EmailStr   # stricter email validation which requires emai-validator to be installed

In [38]:
user1 = User(
    username="johney_tepp",
    age=30,
    email="johney.tepp@example-pet-store.com"
)

print(user1)

username='johney_tepp' age=30 email='johney.tepp@example-pet-store.com'


In [40]:
print(user1.model_json_schema())

{'properties': {'username': {'description': 'Unique username for the user', 'title': 'Username', 'type': 'string'}, 'age': {'description': 'Age should be greater than 18', 'exclusiveMinimum': 18, 'title': 'Age', 'type': 'integer'}, 'email': {'description': 'Email should be a valid email address', 'pattern': '^[\\w\\.-]+@[\\w\\.-]+\\.\\w+$', 'title': 'Email', 'type': 'string'}}, 'required': ['username', 'age', 'email'], 'title': 'User', 'type': 'object'}
